<a href="https://colab.research.google.com/github/riborra/JPI/blob/main/UNICO_DEF_JPI_ALLN5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
## 1. Carga de SOFTWARE: TensorFlow, Keras y librerías Python

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


2.4.1


---
## 2. Carga de DATOS: fichero CSV con 3.676 muestras de datos en formato [ CARACTERÍSTICAS | ETIQUETA ]
* **Características**: CALENDARIO | CLIMA | MEDIOAMBIENTE
>* **CALENDARIO**: FECHA, NDSEM, NSEM, NMES, ANNO, LABOR
>* **CLIMA**: TEMP_MAX, HUMR_MED, PRES_MAX, VELV_MAX, VELV_MED, RAFV_MAX, PREC_TOT
>* **MEDIOAMBIENTE**: NO2_MAX, SO2_MAX, O3_MAX, NOX_MAX, C6H6_ALC, C7H8_ALC, XIL_ALC, PM10_MAX, CO_MAX
* **Etiqueta**: POLENES
>* **AMARANTÁCEA**: AMARA, AMA_N
>* **OLEÁCEA**: OLEAC, OLE_N
>* **POÁCEA**: POACE, POA_N
>* **QUERCUS**: QUERC, QUE_N
>* **TOTAL**: TOTAL, TOT_N

In [2]:
dataset_path = tf.keras.utils.get_file("CCM_ALL_N5.csv", "https://raw.githubusercontent.com/riborra/JPI/main/CCM_ALL_N5.csv")

In [3]:
column_names = ['FECHA','NDSEM','NSEM','NMES','ANNO','LABOR','TEMP_MAX','HUMR_MED','PRES_MAX','VELV_MAX','VELV_MED','RAFV_MAX','PREC_TOT','NO2_MAX','SO2_MAX','O3_MAX','NOX_MAX','C6H6_ALC','C7H8_ALC','XIL_ALC','PM10_MAX','CO_MAX','AMARA','AMA_N','OLEAC','OLE_N','POACE','POA_N','QUERC','QUE_N','TOTAL','TOT_N'] 
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=";", skipinitialspace=True)

dataset = raw_dataset.copy()

---
## 3. Parametrización de Características a procesar por el Modelo y de Etiqueta a predecir
*   caract_CALEN : procesar características de **CALENDARIO** (s/n)
*   caract_CLIMA : procesar características de **CLIMA** (s/n)
*   caract_MEDIO : procesar características de **MEDIOAMBIENTE** (s/n)
*   etiq_AMARA : predecir etiqueta **AMARANTÁCEA** (s/n)
*   etiq_OLEAC : predecir etiqueta **OLEÁCEA** (s/n)
*   etiq_POACE : predecir etiqueta **POÁCEA** (s/n)
*   etiq_QUERC : predecir etiqueta **QUERCUS** (s/n)
*   etiq_TOTAL : predecir etiqueta **TOTAL** (s/n)

In [4]:
modelo_REGRES = False
modelo_CLASIF = True

caract_CALEN = True
caract_CLIMA = True
caract_MEDIO = True

etiq_AMARA = True
etiq_OLEAC = False
etiq_POACE = False
etiq_QUERC = False
etiq_TOTAL = False

---
## 4. Revisión y depuración de los DATOS (CSV con 3.676 muestras)

In [5]:
dataset.head()

,FECHA,NDSEM,NSEM,NMES,ANNO,LABOR,TEMP_MAX,HUMR_MED,PRES_MAX,VELV_MAX,VELV_MED,RAFV_MAX,PREC_TOT,NO2_MAX,SO2_MAX,O3_MAX,NOX_MAX,C6H6_ALC,C7H8_ALC,XIL_ALC,PM10_MAX,CO_MAX,AMARA,AMA_N,OLEAC,OLE_N,POACE,POA_N,QUERC,QUE_N,TOTAL,TOT_N
0,17/06/2009,3,25,6,2009,L,32.6,59.0,1017.0,17.0,3.0,28.0,0.0,33.0,3.0,70.0,49.0,0.4,3.6,2.5,43.0,0.1,2,0,1,0,0,0,0,0,6,0
1,18/06/2009,4,25,6,2009,L,32.9,51.0,1017.0,21.0,3.0,28.0,0.0,29.0,4.0,61.0,39.0,0.5,4.9,3.0,0.0,0.0,2,0,2,0,1,0,0,0,11,0
2,19/06/2009,5,25,6,2009,L,39.7,38.0,1016.0,22.0,3.0,32.0,0.0,52.0,5.0,67.0,81.0,0.5,6.9,2.7,45.0,0.1,8,0,4,0,2,0,2,0,25,0
3,20/06/2009,6,25,6,2009,F,32.3,63.0,1015.0,24.0,4.0,37.0,0.1,10.0,3.0,71.0,15.0,0.4,2.6,2.4,0.0,0.0,0,0,2,0,2,0,2,0,11,0
4,21/06/2009,7,25,6,2009,F,30.3,65.0,1016.0,19.0,3.0,34.0,0.0,4.0,3.0,75.0,7.0,0.4,2.1,2.1,0.0,0.0,2,0,4,0,0,0,1,0,15,0


In [6]:
dataset.tail()

,FECHA,NDSEM,NSEM,NMES,ANNO,LABOR,TEMP_MAX,HUMR_MED,PRES_MAX,VELV_MAX,VELV_MED,RAFV_MAX,PREC_TOT,NO2_MAX,SO2_MAX,O3_MAX,NOX_MAX,C6H6_ALC,C7H8_ALC,XIL_ALC,PM10_MAX,CO_MAX,AMARA,AMA_N,OLEAC,OLE_N,POACE,POA_N,QUERC,QUE_N,TOTAL,TOT_N
3672,27/12/2019,5,52,12,2019,L,18.7,72.9,1029.4,8.4,4.4,14.5,0.0,56.0,2.0,16.0,147.0,0.8,6.6,1.9,47.0,0.4,1,0,0,0,0,0,0,0,31,0
3673,28/12/2019,6,52,12,2019,F,16.2,84.9,1030.8,16.1,5.6,28.2,0.0,39.0,5.0,17.0,92.0,0.8,3.5,1.3,46.0,0.4,0,0,0,0,0,0,0,0,20,0
3674,29/12/2019,7,52,12,2019,F,15.9,83.5,1032.8,9.7,4.0,14.5,0.3,41.0,3.0,22.0,80.0,0.8,2.6,1.3,48.0,0.4,0,0,0,0,0,0,0,0,17,0
3675,30/12/2019,1,53,12,2019,L,15.6,78.3,1033.1,9.7,4.2,16.7,0.0,55.0,2.0,19.0,137.0,0.8,2.5,1.3,41.0,0.4,2,0,0,0,0,0,0,0,29,0
3676,31/12/2019,2,53,12,2019,L,13.8,83.5,1033.7,9.7,5.3,16.1,0.0,48.0,3.0,11.0,118.0,0.9,3.3,1.3,73.0,0.5,0,0,0,0,0,0,0,0,27,0


---
### Eliminación de muestras con valores no válidos

In [7]:
print(dataset.isna().sum())

dataset = dataset.dropna()

FECHA       0
NDSEM       0
NSEM        0
NMES        0
ANNO        0
LABOR       0
TEMP_MAX    0
HUMR_MED    0
PRES_MAX    0
VELV_MAX    0
VELV_MED    0
RAFV_MAX    0
PREC_TOT    0
NO2_MAX     0
SO2_MAX     0
O3_MAX      0
NOX_MAX     0
C6H6_ALC    0
C7H8_ALC    0
XIL_ALC     0
PM10_MAX    0
CO_MAX      0
AMARA       0
AMA_N       0
OLEAC       0
OLE_N       0
POACE       0
POA_N       0
QUERC       0
QUE_N       0
TOTAL       0
TOT_N       0
dtype: int64


---
### Normalización de características (ONE-HOT ENCODING) y aplicación de la parametrización seleccionada para Características (CALEN|CLIMA|MEDIO) y Etiqueta (AMARA|OLEAC|POACE|QUERC|TOTAL)

In [8]:
dataset.pop('FECHA')
ndsem = dataset.pop('NDSEM')
nsem = dataset.pop('NSEM')
nmes = dataset.pop('NMES')
anno = dataset.pop('ANNO')
labor = dataset.pop('LABOR')

if caract_CALEN == True:
  dataset['DS_LUN'] = (ndsem == 1)*1.0
  dataset['DS_MAR'] = (ndsem == 2)*1.0
  dataset['DS_MIE'] = (ndsem == 3)*1.0
  dataset['DS_JUE'] = (ndsem == 4)*1.0
  dataset['DS_VIE'] = (ndsem == 5)*1.0
  dataset['DS_SAB'] = (ndsem == 6)*1.0
  dataset['DS_DOM'] = (ndsem == 7)*1.0
  dataset['NSEM_01'] = (nsem == 1)*1.0
  dataset['NSEM_02'] = (nsem == 2)*1.0
  dataset['NSEM_03'] = (nsem == 3)*1.0
  dataset['NSEM_04'] = (nsem == 4)*1.0
  dataset['NSEM_05'] = (nsem == 5)*1.0
  dataset['NSEM_06'] = (nsem == 6)*1.0
  dataset['NSEM_07'] = (nsem == 7)*1.0
  dataset['NSEM_08'] = (nsem == 8)*1.0
  dataset['NSEM_09'] = (nsem == 9)*1.0
  dataset['NSEM_10'] = (nsem == 10)*1.0
  dataset['NSEM_11'] = (nsem == 11)*1.0
  dataset['NSEM_12'] = (nsem == 12)*1.0
  dataset['NSEM_13'] = (nsem == 13)*1.0
  dataset['NSEM_14'] = (nsem == 14)*1.0
  dataset['NSEM_15'] = (nsem == 15)*1.0
  dataset['NSEM_16'] = (nsem == 16)*1.0
  dataset['NSEM_17'] = (nsem == 17)*1.0
  dataset['NSEM_18'] = (nsem == 18)*1.0
  dataset['NSEM_19'] = (nsem == 19)*1.0
  dataset['NSEM_20'] = (nsem == 20)*1.0
  dataset['NSEM_21'] = (nsem == 21)*1.0
  dataset['NSEM_22'] = (nsem == 22)*1.0
  dataset['NSEM_23'] = (nsem == 23)*1.0
  dataset['NSEM_24'] = (nsem == 24)*1.0
  dataset['NSEM_25'] = (nsem == 25)*1.0
  dataset['NSEM_26'] = (nsem == 26)*1.0
  dataset['NSEM_27'] = (nsem == 27)*1.0
  dataset['NSEM_28'] = (nsem == 28)*1.0
  dataset['NSEM_29'] = (nsem == 29)*1.0
  dataset['NSEM_30'] = (nsem == 30)*1.0
  dataset['NSEM_31'] = (nsem == 31)*1.0
  dataset['NSEM_32'] = (nsem == 32)*1.0
  dataset['NSEM_33'] = (nsem == 33)*1.0
  dataset['NSEM_34'] = (nsem == 34)*1.0
  dataset['NSEM_35'] = (nsem == 35)*1.0
  dataset['NSEM_36'] = (nsem == 36)*1.0
  dataset['NSEM_37'] = (nsem == 37)*1.0
  dataset['NSEM_38'] = (nsem == 38)*1.0
  dataset['NSEM_39'] = (nsem == 39)*1.0
  dataset['NSEM_40'] = (nsem == 40)*1.0
  dataset['NSEM_41'] = (nsem == 41)*1.0
  dataset['NSEM_42'] = (nsem == 42)*1.0
  dataset['NSEM_43'] = (nsem == 43)*1.0
  dataset['NSEM_44'] = (nsem == 44)*1.0
  dataset['NSEM_45'] = (nsem == 45)*1.0
  dataset['NSEM_46'] = (nsem == 46)*1.0
  dataset['NSEM_47'] = (nsem == 47)*1.0
  dataset['NSEM_48'] = (nsem == 48)*1.0
  dataset['NSEM_49'] = (nsem == 49)*1.0
  dataset['NSEM_50'] = (nsem == 50)*1.0
  dataset['NSEM_51'] = (nsem == 51)*1.0
  dataset['NSEM_52'] = (nsem == 52)*1.0
  dataset['NSEM_53'] = (nsem == 53)*1.0
  dataset['NSEM_54'] = (nsem == 54)*1.0
  dataset['MES_ENE'] = (nmes == 1)*1.0
  dataset['MES_FEB'] = (nmes == 2)*1.0
  dataset['MES_MAR'] = (nmes == 3)*1.0
  dataset['MES_ABR'] = (nmes == 4)*1.0
  dataset['MES_MAY'] = (nmes == 5)*1.0
  dataset['MES_JUN'] = (nmes == 6)*1.0
  dataset['MES_JUL'] = (nmes == 7)*1.0
  dataset['MES_AGO'] = (nmes == 8)*1.0
  dataset['MES_SEP'] = (nmes == 9)*1.0
  dataset['MES_OCT'] = (nmes == 10)*1.0
  dataset['MES_NOV'] = (nmes == 11)*1.0
  dataset['MES_DIC'] = (nmes == 12)*1.0
  dataset['ANNO_09'] = (anno == 2009)*1.0
  dataset['ANNO_10'] = (anno == 2010)*1.0
  dataset['ANNO_11'] = (anno == 2011)*1.0
  dataset['ANNO_12'] = (anno == 2012)*1.0
  dataset['ANNO_13'] = (anno == 2013)*1.0
  dataset['ANNO_14'] = (anno == 2014)*1.0
  dataset['ANNO_15'] = (anno == 2015)*1.0
  dataset['ANNO_16'] = (anno == 2016)*1.0
  dataset['ANNO_17'] = (anno == 2017)*1.0
  dataset['ANNO_18'] = (anno == 2018)*1.0
  dataset['ANNO_19'] = (anno == 2019)*1.0
  dataset['D_LABOR'] = (labor == 'L')*1.0
  dataset['D_FESTIV'] = (labor == 'F')*1.0
  
if caract_CLIMA == False:
  dataset.pop('TEMP_MAX')
  dataset.pop('HUMR_MED')
  dataset.pop('PRES_MAX')
  dataset.pop('VELV_MAX')
  dataset.pop('VELV_MED')
  dataset.pop('RAFV_MAX')
  dataset.pop('PREC_TOT')

if caract_MEDIO == False:
  dataset.pop('NO2_MAX')
  dataset.pop('SO2_MAX')
  dataset.pop('O3_MAX')
  dataset.pop('NOX_MAX')
  dataset.pop('C6H6_ALC')
  dataset.pop('C7H8_ALC')
  dataset.pop('XIL_ALC')
  dataset.pop('PM10_MAX')
  dataset.pop('CO_MAX')

if etiq_AMARA == False:
  dataset.pop('AMARA')
  dataset.pop('AMA_N')

if etiq_OLEAC == False:
  dataset.pop('OLEAC')
  dataset.pop('OLE_N')

if etiq_POACE == False:
  dataset.pop('POACE')
  dataset.pop('POA_N')

if etiq_QUERC == False:
  dataset.pop('QUERC')
  dataset.pop('QUE_N')

if etiq_TOTAL == False:
  dataset.pop('TOTAL')
  dataset.pop('TOT_N')

dataset.tail()

,TEMP_MAX,HUMR_MED,PRES_MAX,VELV_MAX,VELV_MED,RAFV_MAX,PREC_TOT,NO2_MAX,SO2_MAX,O3_MAX,NOX_MAX,C6H6_ALC,C7H8_ALC,XIL_ALC,PM10_MAX,CO_MAX,AMARA,AMA_N,DS_LUN,DS_MAR,DS_MIE,DS_JUE,DS_VIE,DS_SAB,DS_DOM,NSEM_01,NSEM_02,NSEM_03,NSEM_04,NSEM_05,NSEM_06,NSEM_07,NSEM_08,NSEM_09,NSEM_10,NSEM_11,NSEM_12,NSEM_13,NSEM_14,NSEM_15,...,NSEM_40,NSEM_41,NSEM_42,NSEM_43,NSEM_44,NSEM_45,NSEM_46,NSEM_47,NSEM_48,NSEM_49,NSEM_50,NSEM_51,NSEM_52,NSEM_53,NSEM_54,MES_ENE,MES_FEB,MES_MAR,MES_ABR,MES_MAY,MES_JUN,MES_JUL,MES_AGO,MES_SEP,MES_OCT,MES_NOV,MES_DIC,ANNO_09,ANNO_10,ANNO_11,ANNO_12,ANNO_13,ANNO_14,ANNO_15,ANNO_16,ANNO_17,ANNO_18,ANNO_19,D_LABOR,D_FESTIV
3672,18.7,72.9,1029.4,8.4,4.4,14.5,0.0,56.0,2.0,16.0,147.0,0.8,6.6,1.9,47.0,0.4,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3673,16.2,84.9,1030.8,16.1,5.6,28.2,0.0,39.0,5.0,17.0,92.0,0.8,3.5,1.3,46.0,0.4,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3674,15.9,83.5,1032.8,9.7,4.0,14.5,0.3,41.0,3.0,22.0,80.0,0.8,2.6,1.3,48.0,0.4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3675,15.6,78.3,1033.1,9.7,4.2,16.7,0.0,55.0,2.0,19.0,137.0,0.8,2.5,1.3,41.0,0.4,2,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3676,13.8,83.5,1033.7,9.7,5.3,16.1,0.0,48.0,3.0,11.0,118.0,0.9,3.3,1.3,73.0,0.5,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


---
### Definición de la **ETIQUETA** (objetivo de la predicción)

* Modelo de **REGRESIÓN**: predicción del valor absoluto de concentración de polen (**AMARA / OLEAC / POACE / QUERC / TOTAL**)
* Modelo de **CLASIFICACIÓN**: predicción del valor discreto *bajo/medio/alto* de concentración de polen (**AMA_N / OLE_N / POA_N / QUE_N / TOT_N**)


In [9]:
if modelo_REGRES == True:
  if etiq_AMARA == True:
    dataset.pop('AMA_N')
  if etiq_OLEAC == True:
    dataset.pop('OLE_N')
  if etiq_POACE == True:
    dataset.pop('POA_N')
  if etiq_QUERC == True:
    dataset.pop('QUE_N')
  if etiq_TOTAL == True:
    dataset.pop('TOT_N')

if modelo_CLASIF == True:
  if etiq_AMARA == True:
    dataset.pop('AMARA')
  if etiq_OLEAC == True:
    dataset.pop('OLEAC')
  if etiq_POACE == True:
    dataset.pop('POACE')
  if etiq_QUERC == True:
    dataset.pop('QUERC')
  if etiq_TOTAL == True:
    dataset.pop('TOTAL')

In [10]:
dataset.tail()

,TEMP_MAX,HUMR_MED,PRES_MAX,VELV_MAX,VELV_MED,RAFV_MAX,PREC_TOT,NO2_MAX,SO2_MAX,O3_MAX,NOX_MAX,C6H6_ALC,C7H8_ALC,XIL_ALC,PM10_MAX,CO_MAX,AMA_N,DS_LUN,DS_MAR,DS_MIE,DS_JUE,DS_VIE,DS_SAB,DS_DOM,NSEM_01,NSEM_02,NSEM_03,NSEM_04,NSEM_05,NSEM_06,NSEM_07,NSEM_08,NSEM_09,NSEM_10,NSEM_11,NSEM_12,NSEM_13,NSEM_14,NSEM_15,NSEM_16,...,NSEM_40,NSEM_41,NSEM_42,NSEM_43,NSEM_44,NSEM_45,NSEM_46,NSEM_47,NSEM_48,NSEM_49,NSEM_50,NSEM_51,NSEM_52,NSEM_53,NSEM_54,MES_ENE,MES_FEB,MES_MAR,MES_ABR,MES_MAY,MES_JUN,MES_JUL,MES_AGO,MES_SEP,MES_OCT,MES_NOV,MES_DIC,ANNO_09,ANNO_10,ANNO_11,ANNO_12,ANNO_13,ANNO_14,ANNO_15,ANNO_16,ANNO_17,ANNO_18,ANNO_19,D_LABOR,D_FESTIV
3672,18.7,72.9,1029.4,8.4,4.4,14.5,0.0,56.0,2.0,16.0,147.0,0.8,6.6,1.9,47.0,0.4,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3673,16.2,84.9,1030.8,16.1,5.6,28.2,0.0,39.0,5.0,17.0,92.0,0.8,3.5,1.3,46.0,0.4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3674,15.9,83.5,1032.8,9.7,4.0,14.5,0.3,41.0,3.0,22.0,80.0,0.8,2.6,1.3,48.0,0.4,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3675,15.6,78.3,1033.1,9.7,4.2,16.7,0.0,55.0,2.0,19.0,137.0,0.8,2.5,1.3,41.0,0.4,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3676,13.8,83.5,1033.7,9.7,5.3,16.1,0.0,48.0,3.0,11.0,118.0,0.9,3.3,1.3,73.0,0.5,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


---
## 5. Subdivisión aleatoria del conjunto de datos en [ APRENDIZAJE | TEST ]
* Muestras para **APRENDIZAJE**: 80%
* Muestras para **TEST**: 20%

In [14]:
if modelo_REGRES == True:
  copia_dataset = dataset.copy(deep=True)
  if etiq_AMARA == True:
    Y = copia_dataset.pop('AMA_N')
  if etiq_OLEAC == True:
    Y = copia_dataset.pop('OLE_N')
  if etiq_POACE == True:
    Y = copia_dataset.pop('POA_N')
  if etiq_QUERC == True:
    Y = copia_dataset.pop('QUE_N')
  X = copia_dataset.copy(deep=True)
  import collections
  from imblearn.over_sampling import SMOTE
  smt = SMOTE()
  #from imblearn.combine import SMOTETomek
  #smt = SMOTETomek(ratio='auto')
  s_X, s_Y = smt.fit_sample(X, Y)
  nuevo_dataset = [s_X, s_Y]
  dataset = nuevo_dataset.copy()
  #print('Original dataset shape:', collections.Counter(Y))
  #print('Resample dataset shape:', collections.Counter(s_Y))
  #normed_train_data = s_normed_train_data
  #train_labels = s_train_labels
  

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

---
### Identificación de la ETIQUETA objetivo en los subconjuntos de datos de APRENDIZAJE y TEST

In [ ]:
if modelo_REGRES == True:
  if etiq_AMARA == True:
    train_labels = train_dataset.pop('AMARA')
    test_labels = test_dataset.pop('AMARA')
  if etiq_OLEAC == True:
    train_labels = train_dataset.pop('OLEAC')
    test_labels = test_dataset.pop('OLEAC')
  if etiq_POACE == True:
    train_labels = train_dataset.pop('POACE')
    test_labels = test_dataset.pop('POACE')
  if etiq_QUERC == True:
    train_labels = train_dataset.pop('QUERC')
    test_labels = test_dataset.pop('QUERC')
  if etiq_TOTAL == True:
    train_labels = train_dataset.pop('TOTAL')
    test_labels = test_dataset.pop('TOTAL')

if modelo_CLASIF == True:
  if etiq_AMARA == True:
    train_labels = train_dataset.pop('AMA_N')
    test_labels = test_dataset.pop('AMA_N')
  if etiq_OLEAC == True:
    train_labels = train_dataset.pop('OLE_N')
    test_labels = test_dataset.pop('OLE_N')
  if etiq_POACE == True:
    train_labels = train_dataset.pop('POA_N')
    test_labels = test_dataset.pop('POA_N')
  if etiq_QUERC == True:
    train_labels = train_dataset.pop('QUE_N')
    test_labels = test_dataset.pop('QUE_N')
  if etiq_TOTAL == True:
    train_labels = train_dataset.pop('TOT_N')
    test_labels = test_dataset.pop('TOT_N')

---
### Normalización y Estandarización de las Características (distribución normal estándar de media=0 y varianza=1)

In [ ]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
normed_train_stats = normed_train_data.describe()
normed_train_stats = normed_train_stats.transpose()
normed_train_stats

In [ ]:
normed_train_data.head()

In [ ]:
normed_train_data.tail()

---
## 6. Definición del Modelo de Red Neuronal
* **Arquitectura**: Capas, Nodos y Función de Activación
* **Hiperparámetros**: Optimizador, Métrica y Función de Pérdida

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

if modelo_REGRES == True:
  def build_model():  
    model = Sequential()
    model.add(Dense(128, activation='sigmoid', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(128, activation='sigmoid'))
    model.add(Dense(1))
    return model

if modelo_CLASIF == True:
  def build_model():  
    model = Sequential()
    model.add(Dense(128, activation='sigmoid', input_shape=[len(train_dataset.keys())]))
    model.add(Dense(128, activation='sigmoid'))
    model.add(Dense(3, activation='softmax'))
    return model

model = build_model()
model.summary()

In [ ]:
if modelo_REGRES == True:
  model.compile(optimizer='adam', loss='huber', metrics=['mse','mae'])

if modelo_CLASIF == True:
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

---
## 7. Aprendizaje del Modelo
* Subdivisión aleatoria de los datos de APRENDIZAJE en 2 subconjuntos de datos [ ENTRENAMIENTO | VALIDACION ]
>* **ENTRENAMIENTO** con el 75% de los datos de APRENDIZAJE
>* **VALIDACION** con el 25% de los datos de APRENDIZAJE

In [ ]:
if modelo_REGRES == True:
  EPOCHS = 100
  history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.25, verbose=1)

if modelo_CLASIF == True:
  EPOCHS = 100
  #class_weights = {0: 0.8, 1: 1.3, 2: 1.1}
  #[0.39210982 3.93842035 5.10763889]
  #from sklearn.utils import class_weight
  #class_weights = class_weight.compute_class_weight('balanced', np.unique(train_labels), train_labels)
  #print(train_labels)
  #print(class_weights)
  #history = model.fit(normed_train_data, train_labels, class_weight=class_weights, epochs=EPOCHS, validation_split = 0.25, verbose=1)

  #import collections
  #from imblearn.over_sampling import SMOTE
  #smt = SMOTE()
  #from imblearn.combine import SMOTETomek
  #smt = SMOTETomek(ratio='auto')
  #s_normed_train_data, s_train_labels = smt.fit_sample(normed_train_data, train_labels)
  #print('Original dataset shape:', collections.Counter(train_labels))
  #print('Resample dataset shape:', collections.Counter(s_train_labels))
  #normed_train_data = s_normed_train_data
  #train_labels = s_train_labels
  
  history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.25, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

---
## 8. Evaluación del Aprendizaje del Modelo
* Detección de **Overfitting** (Sobreajuste): el Modelo 'aprende' en exceso las características de los datos de Entrenamiento (cuyos resultados mejoran en cada iteración), lo que a cambio provoca el empeoramiento de los resultados para los datos de Validación, reflejado en el aumento de su Función de Pérdida

---
### Gráfico de evolución del Aprendizaje para la **Métrica**, comparando datos de Entrenamiento y Validación

In [ ]:
if modelo_REGRES == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Métrica MSE')
    plt.plot(hist['epoch'], hist['mse'],'r--', label='Métrica MSE Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_mse'],'b', label = 'Métrica MSE Datos Validación')
    plt.ylim([0,110])
    plt.legend()
    plt.show()

if modelo_CLASIF == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Precisión')
    plt.plot(hist['epoch'], hist['accuracy'],'r--', label='Precisión Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_accuracy'],'b', label = 'Precisión Datos Validación')
    plt.ylim([0,1])
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

---
### Gráfico de evolución del Aprendizaje para la **Función de Pérdida**, comparando datos de Entrenamiento y Validación

In [ ]:
if modelo_REGRES == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Función de Pérdida')
    plt.plot(hist['epoch'], hist['loss'],'r--', label='Función de Pérdida Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_loss'],'b', label = 'Función de Pérdida Datos Validación')
    plt.ylim([0,5])
    plt.legend()
    plt.show()

if modelo_CLASIF == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Función de Pérdida')
    plt.plot(hist['epoch'], hist['loss'],'r--', label='Función de Pérdida Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_loss'],'b', label = 'Función de Pérdida Datos Validación')
    plt.ylim([0,1])
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

---
## 9. Optimización del Modelo
* Corrección de **Overfitting** (Sobreajuste): al detectar que los resultados de los datos de Validación empiezan a empeorar, se detiene automáticamente el proceso de aprendizaje (**Early Stopping**)

In [ ]:
model = build_model()

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss', patience=7)

if modelo_REGRES == True:
  model.compile(optimizer='adam', loss='huber', metrics=['mse','mae'])
  history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.25, verbose=1, callbacks=[early_stop])

if modelo_CLASIF == True:
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, class_weight=class_weights, validation_split = 0.25, verbose=1, callbacks=[early_stop])
  history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.25, verbose=1, callbacks=[early_stop])

---
### Gráfico de evolución del Aprendizaje para la **Métrica**, comparando datos de Entrenamiento y Validación, aplicando **Early Stopping** para prevenir el Sobreajuste

In [ ]:
if modelo_REGRES == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Métrica MSE')
    plt.plot(hist['epoch'], hist['mse'],'r--', label='Métrica MSE Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_mse'],'b', label = 'Métrica MSE Datos Validación')
    plt.ylim([0,110])
    plt.legend()
    plt.show()

if modelo_CLASIF == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Precisión')
    plt.plot(hist['epoch'], hist['accuracy'],'r--', label='Precisión Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_accuracy'],'b', label = 'Precisión Datos Validación')
    plt.ylim([0,1])
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

---
### Gráfico de evolución del Aprendizaje para la **Función de Pérdida**, comparando datos de Entrenamiento y Validación, aplicando **Early Stopping** para prevenir el Sobreajuste

In [ ]:
if modelo_REGRES == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Función de Pérdida')
    plt.plot(hist['epoch'], hist['loss'],'r--', label='Función de Pérdida Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_loss'],'b', label = 'Función de Pérdida Datos Validación')
    plt.ylim([0,5])
    plt.legend()
    plt.show()

if modelo_CLASIF == True:
  def plot_history(history):     
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Función de Pérdida')
    plt.plot(hist['epoch'], hist['loss'],'r--', label='Función de Pérdida Datos Entrenamiento')
    plt.plot(hist['epoch'], hist['val_loss'],'b', label = 'Función de Pérdida Datos Validación')
    plt.ylim([0,1])
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

---
## 10. Evaluación Final del Modelo
* Evaluación de la **capacidad predictiva real del Modelo**, utilizando el subconjunto de datos de **Test**, que fueron separados del conjunto de datos original antes de iniciar el proceso de Aprendizaje
* Datos Originales
>* 80% = **APRENDIZAJE** (ENTRENAMIENTO + VALIDACION)
>* 20% = **TEST**

---
### **Modelo de REGRESIÓN**: obtención del **Error Cuadrático Medio** (MSE) y del **Error Absoluto Medio** (NAE) de la predicción para el subconjunto de Datos de Test

In [ ]:
if modelo_REGRES == True:
  loss, mse, mae = model.evaluate(normed_test_data, test_labels)
  print("\nERROR CUADRÁTICO MEDIO del conjunto de datos de TEST: {:5.2f} granos/metro cúbico".format(mse))
  print("\nERROR ABSOLUTO MEDIO del conjunto de datos de TEST: {:5.2f} granos/metro cúbico".format(mae))



---
### **Modelo de CLASIFICACIÓN**: obtención de la **Exactitud** de la predicción, así como de la **Matriz de Confusión** y de los indicadores de **Precisión**, **Exhaustividad** y **F1** para el subconjunto de Datos de Test

In [ ]:
if modelo_CLASIF == True:
  loss, accuracy = model.evaluate(normed_test_data, test_labels)
  print("\nACCURACY/EXACTITUD del conjunto de datos de TEST: {:5.2f}%".format(accuracy*100))

In [ ]:
if modelo_CLASIF == True:
  def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de Confusión',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:5.2f}".format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('Observación')
    plt.xlabel('Predicción')

  from collections import Counter
  from sklearn.metrics import confusion_matrix
  import itertools

  # Predict the values from the TEST dataset
  Y_pred = model.predict(normed_test_data)
  #print(Y_pred)

  # Convert predictions classes to one hot vectors 
  Y_pred_classes = np.argmax(Y_pred, axis = 1) 
  #print(Y_pred_classes)

  #print(test_labels)

  # Convert TEST observations to one hot vectors
  #Y_true = np.argmax(test_labels, axis = None) 
  Y_true = test_labels.to_numpy()
  #print(Y_true)

  # compute the confusion matrix
  confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

  # plot the confusion matrix
  plot_confusion_matrix(confusion_mtx, classes = range(3))

  # Accuracy (EXACTITUD = Aciertos totales sobre total de muestras)
  from sklearn.metrics import accuracy_score
  print("\nACCURACY/EXACTITUD..: ",accuracy_score(Y_true, Y_pred_classes))

  # Precision (PRECISION = Aciertos de X sobre total de predicciones de X)
  from sklearn.metrics import precision_score
  print("\nPRECISION/PRECISION.: ",precision_score(Y_true, Y_pred_classes, average=None))

  # Recall (EXHAUSTIVIDAD = Aciertos de X sobre total de casos reales de X)
  from sklearn.metrics import recall_score
  print("RECALL/EXHAUSTIVIDAD: ",  recall_score(Y_true, Y_pred_classes, average=None))

  # F1
  from sklearn.metrics import f1_score
  print("F1..................: ",f1_score(Y_true, Y_pred_classes, average=None))
  print("\n")